## Notebook for making predictions with deleted constant y samples


In [14]:
import pandas as pd
import sys
from sklearn.model_selection import train_test_split
from catboost import CatBoostRegressor
import numpy as np
from sklearn.ensemble import StackingRegressor
import lightgbm as lgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.ensemble import BaggingRegressor


### Reading dataset stored in other notebooks

In [15]:
sys.path.append('../')

%store -r dml

dm = dml

## Getting data

In [16]:
print(dm.data_A.columns)

X_A_train = dm.data_A.iloc[:, 2:]
Y_A_train = dm.data_A.iloc[:, 1]

X_B_train = dm.data_B.iloc[:, 2:]
Y_B_train = dm.data_B.iloc[:, 1]

X_C_train = dm.data_C.iloc[:, 2:]
Y_C_train = dm.data_C.iloc[:, 1]

X_A_test = dm.X_test_estimated_a[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_B_test = dm.X_test_estimated_b[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]
X_C_test = dm.X_test_estimated_c[dm.X_test_estimated_a.columns.intersection(X_A_train.columns)]

#X_A_train['direct_rad:W_lag_12']

Index(['date_forecast', 'pv_measurement', 'is_day:idx', 'is_in_shadow:idx',
       'precip_type_5min:idx', 'clear_sky_energy_1h:J', 'diffuse_rad_1h:J',
       'direct_rad_1h:J', 'fresh_snow_12h:cm', 'fresh_snow_24h:cm',
       'precip_5min:mm', 'rain_water:kgm2', 'snow_water:kgm2',
       'super_cooled_liquid_water:kgm2', 'absolute_humidity_2m:gm3',
       'air_density_2m:kgm3', 'clear_sky_rad:W', 'dew_point_2m:K',
       'diffuse_rad:W', 'direct_rad:W', 'effective_cloud_cover:p',
       'msl_pressure:hPa', 'pressure_100m:hPa', 'pressure_50m:hPa',
       'relative_humidity_1000hPa:p', 'sfc_pressure:hPa', 'sun_azimuth:d',
       'sun_elevation:d', 't_1000hPa:K', 'total_cloud_cover:p', 'visibility:m',
       'wind_speed_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_v_10m:ms',
       'cloud_base_agl:m', 'month', 'hours', 'week_number', 'day_year',
       'location'],
      dtype='object')


In [17]:
# remove special characters from column names
X_A_train.columns = X_A_train.columns.str.replace(':', '_')
X_B_train.columns = X_B_train.columns.str.replace(':', '_')
X_C_train.columns = X_C_train.columns.str.replace(':', '_')

X_A_test.columns = X_A_test.columns.str.replace(':', '_')
X_B_test.columns = X_B_test.columns.str.replace(':', '_')
X_C_test.columns = X_C_test.columns.str.replace(':', '_')

In [18]:
X_train = pd.concat([X_A_train, X_B_train, X_C_train])
Y_train = pd.concat([Y_A_train, Y_B_train, Y_C_train])

X_test = pd.concat([X_A_test, X_B_test, X_C_test])

# Stack model

In [19]:
catModel = CatBoostRegressor(**{'objective': 'MAE', 'colsample_bylevel': 0.08274297094205027, 'depth': 13, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'iterations': 4134, 'l2_leaf_reg': 2, 'subsample': 0.9964148785726921})

lgbmModel = lgb.LGBMRegressor(**{'learning_rate': 0.0010270953806470696, 'num_leaves': 378, 'subsample': 0.5052366920183552, 'colsample_bytree': 0.6645498697580586, 'min_data_in_leaf': 40})

nnModel = MLPRegressor()

bagModel = BaggingRegressor()

metaModel = LinearRegression()

In [20]:
# stacking classifier
stackingModel = StackingRegressor(
    estimators=[('cat', catModel), ('lgbm', lgbmModel)],
    final_estimator= metaModel
)


In [21]:
# fit stacking models
stackingModel.fit(X_train, Y_train)


0:	learn: 321.2359608	total: 226ms	remaining: 15m 34s
1:	learn: 316.8550150	total: 320ms	remaining: 11m 2s
2:	learn: 311.1699398	total: 327ms	remaining: 7m 30s
3:	learn: 307.1628432	total: 402ms	remaining: 6m 54s
4:	learn: 303.3512479	total: 410ms	remaining: 5m 38s
5:	learn: 300.1647435	total: 418ms	remaining: 4m 47s
6:	learn: 293.2378384	total: 499ms	remaining: 4m 54s
7:	learn: 289.8710291	total: 586ms	remaining: 5m 2s
8:	learn: 286.4524992	total: 674ms	remaining: 5m 8s
9:	learn: 283.2074691	total: 776ms	remaining: 5m 20s
10:	learn: 280.1213902	total: 856ms	remaining: 5m 20s
11:	learn: 280.1213902	total: 861ms	remaining: 4m 55s
12:	learn: 274.0443601	total: 868ms	remaining: 4m 35s
13:	learn: 271.2177290	total: 878ms	remaining: 4m 18s
14:	learn: 264.8867622	total: 970ms	remaining: 4m 26s
15:	learn: 264.8867622	total: 975ms	remaining: 4m 10s
16:	learn: 262.5341575	total: 980ms	remaining: 3m 57s
17:	learn: 255.6210699	total: 1.01s	remaining: 3m 51s
18:	learn: 255.6210699	total: 1.01s	rem

StackingRegressor(estimators=[('cat',
                               <catboost.core.CatBoostRegressor object at 0x0000025D566736A0>),
                              ('lgbm',
                               LGBMRegressor(colsample_bytree=0.6645498697580586,
                                             learning_rate=0.0010270953806470696,
                                             min_data_in_leaf=40,
                                             num_leaves=378,
                                             subsample=0.5052366920183552))],
                  final_estimator=LinearRegression())

## Preds

In [28]:
# preds for stacking models
stacking_preds = pd.DataFrame(stackingModel.predict(X_test))

[LightGBM] [Warning] min_data_in_leaf is set=40, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=40


In [23]:
#cat_preds = pd.read_csv("./ext_preds/bestpred.csv")

In [29]:
stacking_preds[stacking_preds < 0] = 0
#pred_mean = np.mean([cat_preds, stacking_preds], axis=0)
#pred_mean[pred_mean < 0] = 0


In [30]:
stacking_preds.to_csv("./ext_preds/cat_lgb.csv")